<img src='https://www.anadronestarting.com/wp-content/uploads/intel-main_opt.png' width=50%>

# 모바일넷을 이용한 이미지분류
<font size=5><b>(Image Classification using Mobilenet)<b></font>

<div align='right'>성  민  석 (Minsuk Sung)</div>
<div align='right'>이  인  구 (Ike Lee)</div>    

<img src='https://chaosmail.github.io/images/deep-learning/classification.png' width=60%>

---

<h1>강의목차<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#필요한-라이브러리-및-옵션" data-toc-modified-id="필요한-라이브러리-및-옵션-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>필요한 라이브러리 및 옵션</a></span><ul class="toc-item"><li><span><a href="#기본-라이브러리(Library)" data-toc-modified-id="기본-라이브러리(Library)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>기본 라이브러리(Library)</a></span></li><li><span><a href="#옵션(Option)" data-toc-modified-id="옵션(Option)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>옵션(Option)</a></span></li></ul></li><li><span><a href="#예제---VOC2012" data-toc-modified-id="예제---VOC2012-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>예제 - VOC2012</a></span><ul class="toc-item"><li><span><a href="#VOC-2012란?" data-toc-modified-id="VOC-2012란?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>VOC 2012란?</a></span></li><li><span><a href="#VOC-데이터의-구성" data-toc-modified-id="VOC-데이터의-구성-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>VOC 데이터의 구성</a></span></li><li><span><a href="#다운로드-링크" data-toc-modified-id="다운로드-링크-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>다운로드 링크</a></span></li><li><span><a href="#학습에-필요한-상수" data-toc-modified-id="학습에-필요한-상수-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>학습에 필요한 상수</a></span></li><li><span><a href="#모델-컴파일" data-toc-modified-id="모델-컴파일-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>모델 컴파일</a></span></li><li><span><a href="#모델-학습하기" data-toc-modified-id="모델-학습하기-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>모델 학습하기</a></span></li><li><span><a href="#모델-저장하기" data-toc-modified-id="모델-저장하기-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>모델 저장하기</a></span></li><li><span><a href="#모델-평가하기" data-toc-modified-id="모델-평가하기-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>모델 평가하기</a></span></li><li><span><a href="#테스트해보기" data-toc-modified-id="테스트해보기-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>테스트해보기</a></span></li></ul></li><li><span><a href="#참고" data-toc-modified-id="참고-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>참고</a></span></li></ul></div>

## 필요한 라이브러리 및 옵션

### 기본 라이브러리(Library)

In [ ]:
import os
import sys
import glob
import random
import itertools
from pathlib import Path
from tqdm import tqdm
import pickle

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
import cv2
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from IPython.display import SVG
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, ElementTree

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.datasets import mnist,cifar10
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.applications import MobileNet, MobileNetV2
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Conv2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy
from tensorflow.keras.callbacks import Callback
from tensorflow.python.client import device_lib

### 옵션(Option)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%matplotlib inline
print(device_lib.list_local_devices())
keras.backend.tensorflow_backend._get_available_gpus()

---

## 예제 - VOC2012

![](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/pascal2.png)

### VOC 2012란?
VOC2012 : Visual Object Classes Challenge 2012 (VOC2012)

### VOC 데이터의 구성
PASCAL VOC Dataset을 다운받아 압축을 풀면 다음과 같은 구조를 확인할 수 있습니다.
```
VOC2012
├── Annotations
│   ├── 2010_000002.xml
│   ├── 2010_000003.xml
│   ├── 2011_000002.xml
│   └── ...
├── ImageSets
│   ├── Action
│   ├── Layout
│   ├── Main
│   └── Segmentation
├── JPEGImages
│   ├── 2010_000002.jpg
│   ├── 2010_000003.jpg
│   ├── 2011_000002.jpg
│   └── ...
├── SegmentationClass
│   ├── 2010_000002.png
│   ├── 2010_000003.png
│   └── 2011_000003.png
└── SegmentationObject
    ├── 2010_000002.png
    ├── 2010_000003.png
    └── ...
```

- Annotations : JPEGImages 폴더 속 원본 이미지와 같은 이름들의 xml파일들이 존재합니다. Object Detection을 위한 정답 데이터이 됩니다.

- ImageSets : 어떤 이미지 그룹을 test, train, trainval, val로 사용할 것인지, 특정 클래스가 어떤 이미지에 있는지 등에 대한 정보들을 포함하고 있는 폴더입니다.

- JPEGImages : jpg확장자를 가진 이미지 파일들이 모여있는 폴더입니다. Object Detection에서 입력 데이터가 됩니다.

- SegmentationClass : Semantic segmentation을 학습하기 위한 label 이미지입니다.

- SegmentationObject : Instance segmentation을 학습하기 위한 label 이미지입니다.

Object Detection을 할 때는 주로 Annotations, JPEGImages폴더가 사용됩니다. 모델에 입력으로 넣는 입력데이터인 경우 그냥 load 해서 사용하면 되나, 지도학습에 핵심이 되는 정답 데이터의 경우는 parsing이 필요한 경우가 있으므로 Annotations의 xml 구조는 잘 알아두는 것이 중요합니다.

출처 : https://deepbaksuvision.github.io/Modu_ObjectDetection/posts/02_01_PASCAL_VOC.html

### 다운로드 링크
- http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
- https://pjreddie.com/projects/pascal-voc-dataset-mirror/

> VOC 2012 데이터는 한 이미지 내에 여러가지 객체가 존재합니다. 그래서 기존의 방법과는 조금 다르게 접근해야합니다.  
아래 코드를 따라가봅시다.

In [ ]:
# 우리가 분류할 20개의 클래스

CLASSES = ['person',  # Person
           'bird', 'cat', 'cow', 'dog', 'horse', 'sheep', # Animal
           'aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train', # Vehicle
           'bottle', 'chair', 'dining table', 'potted plant', 'sofa', 'tv monitor' # Indoor
          ]

### 학습에 필요한 상수

In [ ]:
# 학습에 필요한 상수들을 정의 합니다. 

IMG_SHAPE = (224, 224, 3)
LEARNING_RATE = 2e-5
BATCH_SIZE = 4
EPOCHS = 3

# 각 이미지의 기본 주소
BASE_PATH = './data/VOC2012/JPEGImages/'
images_dir = Path(BASE_PATH).expanduser()
print(images_dir)

# 각 이미지별 클래스의 기본 주소
XML_BASE_PATH = './data/VOC2012/Annotations/'
annotations_dir = Path(XML_BASE_PATH).expanduser()
print(annotations_dir)

In [ ]:
#입력 데이터를 렌덤하게 받아서 데이터 포멧을 확인 합니다. 

flist = glob.glob(BASE_PATH+'*.jpg')
idx = np.random.randint(0,len(flist))
fpath = flist[idx]
img = cv2.imread(fpath)
fname = os.path.basename(fpath).split('.')[0]

image = Image.open(fpath).convert("RGB")
draw = ImageDraw.Draw(image)

xml = open(XML_BASE_PATH+fname+'.xml', "r")
tree = ET.parse(xml)
root = tree.getroot()

size = root.find("size")

width = size.find("width").text
height = size.find("height").text
channels = size.find("depth").text

print('-'*50)
print("Image properties")
print('-'*50)
print('File : {}'.format(fpath))
print("width : {}\nheight : {}\nchannels : {}\n".format(width, height, channels))
plt.figure(figsize=(15,12))
plt.imshow(image)
plt.show()

objects = root.findall("object")
print('-'*50)
print("Objects Description")
print('-'*50)
for _object in objects:
    name = _object.find("name").text
    bndbox = _object.find("bndbox")
    xmin = int(bndbox.find("xmin").text)
    ymin = int(bndbox.find("ymin").text)
    xmax = int(bndbox.find("xmax").text)
    ymax = int(bndbox.find("ymax").text)
    
    # Box를 그릴 때, 왼쪽 상단 점(xmin,ymin)과, 오른쪽 하단 점의 좌표(xmax,ymax)를 입력으로 주면 됩니다.
    draw.rectangle(((xmin, ymin), (xmax, ymax)), outline="yellow",width=3)
    draw.text((xmin, ymin-10), name,align='center')

    print("class : {:3}\nxmin : {:3}, ymin : {:3}\nxmax : {:3}, ymax : {:3}\n".format(name, xmin, ymin, xmax, ymax))

plt.figure(figsize=(15,12))
plt.imshow(image)
plt.show()

In [ ]:
# 데이터의 레이블을 정리 합니다 

def xml_to_labels(xml_data, unique_labels):
    root = ET.XML(xml_data)
    labels = set() if unique_labels else []
    labels_add = labels.add if unique_labels else labels.append # speeds up method lookup
    for i, child in enumerate(root):
        if child.tag == 'filename':
            img_filename = child.text
        if child.tag == 'object':
            for subchild in child:
                if subchild.tag == 'name':
                    labels_add(subchild.text)
    return img_filename, list(labels)

def get_labels(annotations_dir, unique_labels=True):
    for annotation_file in annotations_dir.iterdir():
        with open(annotation_file) as f:
            yield xml_to_labels(f.read(), unique_labels)

In [ ]:
# img_metadata = pd.DataFrame(get_labels(annotations_dir), columns=['filename', 'labels'])
# print('Found {} images'.format(len(img_metadata)))
# img_metadata.sample(5)

In [ ]:
# all_labels = [label for lbs in img_metadata['labels'] for label in lbs]
# labels_count = Counter(all_labels)
# ax = sns.countplot(all_labels, order=[k for k, _ in labels_count.most_common()], log=True)
# ax.set_title('Number of images with a class label')
# ax.set_ylim(1E2, 1E4)
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# plt.show()

In [ ]:
# height, width, size = np.empty(len(img_metadata)), np.empty(len(img_metadata)), np.empty(len(img_metadata)) 
# for i, img_filepath in img_metadata['filename'].iteritems():
#     w, h = Image.open(images_dir.joinpath(img_filepath)).size
#     width[i], height[i], size[i] = w, h, w * h * 3 * 1E-6
# plt.scatter(width, height, alpha=0.5)
# plt.xlabel('Width'); plt.ylabel('Height'); plt.show()
# plt.hist(size, bins=50, log=True)
# plt.xlabel('Image size (MB)');
# plt.ylabel('Count');

In [ ]:
# 우리가 모델 훈련에 사용할 데이터 내용 입니다.

class_list = []
num_list = []

IMAGE_BASE_PATH = './data/VOC4IC/'
train_path = IMAGE_BASE_PATH + 'train/'
for folder in os.listdir(train_path):
    folder_size = len(os.listdir(train_path+folder))
#     print('{:<15} : {}'.format(folder,folder_size))
    class_list.append(folder)
    num_list.append(folder_size)
    
voc_s = pd.Series(num_list,index=class_list)
voc_s.sort_values().plot(kind='bar')
plt.show()

print(voc_s.sort_values())

In [ ]:
# 우리가 모델 훈련 검증에 사용할 데이터 내용 입니다.

class_list = []
num_list = []

IMAGE_BASE_PATH = './data/VOC4IC/'
valid_path = IMAGE_BASE_PATH + 'valid/'
for folder in os.listdir(valid_path):
    folder_size = len(os.listdir(valid_path+folder))
#     print('{:<15} : {}'.format(folder,folder_size))
    class_list.append(folder)
    num_list.append(folder_size)
    
voc_s = pd.Series(num_list,index=class_list)
voc_s.sort_values().plot(kind='bar')
plt.show()

print(voc_s.sort_values())

In [ ]:
# 우리가 훈련된 모델을 검증하는데 사용할 데이터 내용 입니다.

class_list = []
num_list = []

IMAGE_BASE_PATH = './data/VOC4IC/'
test_path = IMAGE_BASE_PATH + 'test/'
for folder in os.listdir(test_path):
    folder_size = len(os.listdir(test_path+folder))
#     print('{:<15} : {}'.format(folder,folder_size))
    class_list.append(folder)
    num_list.append(folder_size)
    
voc_s = pd.Series(num_list,index=class_list)
voc_s.sort_values().plot(kind='bar')
plt.show()

print(voc_s.sort_values())

In [ ]:
# base model 의 input shape, 그리고  trainable 을 false 로 합니다. 

base_model = MobileNetV2(input_shape=(224,224,3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

In [ ]:
# transfer learning에서 마지막 3개 block 을 사용하여 모델을 트레이닝 합니다. 

set_trainable = False
for layer in tqdm(base_model.layers):
    if layer.name in ['block_14_expand','block_15_expand', 'block_16_expand']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# 레이어 구성을 살펴 봅니다. 

layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

In [ ]:
# 모델 만들기, 기존 모델의 weight 값을 사용하고 pooling 과 activation 함수를 추가 합니다. 

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512,activation='relu'),
    Dense(64,activation='relu'),
    Dense(len(CLASSES), activation='softmax')
])
model.summary()

plot_model(model,to_file='./img/model/voc2012_mobilenet_model.png',show_shapes=True)

### 모델 컴파일

In [ ]:
model.compile(loss=categorical_crossentropy, 
              optimizer=Adam(learning_rate=0.0001), # transfer learning 여기서 학습률을 더 작게 
              metrics=['acc'])

### 모델 학습하기

In [ ]:
# 트레인 데이터 augmentation 의로 데이터를 증가 시킴니다.  

train_datagen = ImageDataGenerator(rotation_range=30,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                   )

train_set = train_datagen.flow_from_directory(IMAGE_BASE_PATH + 'train/',
                                              target_size=(224, 224),
                                              batch_size=8,
                                              shuffle=True,
                                              class_mode='categorical')

In [ ]:
# 학습중에 validation에 사용할 데이터셋 입니다.  

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_set = valid_datagen.flow_from_directory(IMAGE_BASE_PATH + 'valid/',
                                            target_size=(224, 224),
                                            batch_size=8,
                                            shuffle=True,
                                            class_mode='categorical')

In [ ]:
# 학습을 완료후에 모델 성능 테스트에 사용할 데이터셋 입니다. 

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(IMAGE_BASE_PATH + 'test/',
                                            target_size=(224, 224),
                                            batch_size=8,
                                            shuffle=True,
                                            class_mode='categorical')

In [ ]:
# 모델 트레이닝 입니다. 

history = model.fit_generator(train_set,
                              steps_per_epoch=train_set.n // train_set.batch_size,
                              epochs=1,
                              validation_data=valid_set,
                              validation_steps=valid_set.n // valid_set.batch_size,
#                               use_multiprocessing=True,
                              )

### 모델 저장하기

In [ ]:
model.save('./bin/mobilenetv2_class20.h5')

In [ ]:
test_set.class_indices.items()

In [ ]:
# key 와 value 값을 바꾸어 줍니다. 
class20 = dict()
for key,value in test_set.class_indices.items():
    class20[value] = key

with open('./bin/class20.pickle', 'wb') as f:
    pickle.dump(class20, f)

In [ ]:
class20

### 모델 평가하기

In [ ]:
# 트레인 데이터와 테스트 데이터 셋으로 loss 와 accuracy 측정합니다.  

train_loss, train_acc = model.evaluate_generator(train_set)
print('Train Loss : {}'.format(train_loss))
print('Train Accuracy : {}'.format(train_acc))

test_loss, test_acc = model.evaluate_generator(test_set)
print('Test Loss : {}'.format(test_loss))
print('Test Accuracy : {}'.format(test_acc))

In [ ]:
# loss 측정값의 시각화 입니다.  

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,label='Training Loss')
plt.plot(epochs,val_loss,label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# accuracy 측정값의 시각화 입니다.  

acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(1,len(loss)+1)

plt.plot(epochs,acc,label='Training Accuarcy')
plt.plot(epochs,val_acc,label='Validation Accuarcy')
plt.title('Training and Validation Accuarcy')
plt.xlabel('Epochs')
plt.ylabel('Accuarcy')
plt.legend()

plt.show()

### 테스트해보기

In [ ]:
# 추론하기 위한 작업입니다. 모델 설정, 입력 데이터 전처리.

def predict_test_img(path):
    img = cv2.imread(path) 
    
    model = load_model('./bin/mobilenetv2_class20.h5')
    
    print('Original Shape : ',img.shape)
    
    img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    print('Resized Shape : ',img.shape)
    plt.imshow(img)
    plt.show()
    
    ans = model.predict_classes(np.expand_dims(img,axis=0))
    with open('./bin/class20.pickle','rb') as f:
        class20 = pickle.load(f)
    print('Predict : {}'.format(class20[ans[0]]))
    
    predicted_result = model.predict(np.expand_dims(img,axis=0))

    pd.DataFrame(predicted_result,columns=class20.values()).iloc[0].plot(kind='bar')
    plt.show()

In [ ]:
# 기존에 사용되지 않았던 임의 데이터를 추론 하기

predict_test_img('img/test/dog.jpg')

In [ ]:
predict_test_img('img/test/boat.jpg')

In [ ]:
predict_test_img('img/test/cat.jpg')

In [ ]:
predict_test_img('img/test/sofa.jpg')

In [ ]:
predict_test_img('img/test/aeroplane.jpg')

In [ ]:
predict_test_img('img/test/person_bike.jpg')

---

## 참고

- Intel OpenVINO
    - https://software.intel.com/en-us/openvino-toolkit
- Tensorflow
    - https://www.tensorflow.org/?hl=ko
- Keras
    - https://keras.io/
    - https://tensorflow.blog/2019/03/06/tensorflow-2-0-keras-api-overview/
    - https://tykimos.github.io/2017/02/22/Integrating_Keras_and_TensorFlow/
    - https://tykimos.github.io/2017/03/08/CNN_Getting_Started/
    - https://raw.githubusercontent.com/keras-team/keras-docs-ko/master/sources/why-use-keras.md
- Keras to Caffe
     - https://github.com/uhfband/keras2caffe
     - http://www.deepvisionconsulting.com/from-keras-to-caffe/
- Fully Connected Layer
    - https://sonofgodcom.wordpress.com/2018/12/31/cnn%EC%9D%84-%EC%9D%B4%ED%95%B4%ED%95%B4%EB%B3%B4%EC%9E%90-fully-connected-layer%EB%8A%94-%EB%AD%94%EA%B0%80/
- Convultional Nueral Network
    - http://aikorea.org/cs231n/convolutional-networks/
    - http://cs231n.stanford.edu/
- CNN Models
    - https://ratsgo.github.io/deep%20learning/2017/10/09/CNNs/

- VOC2012
    - https://blog.godatadriven.com/rod-keras-multi-label
    - https://gist.github.com/rragundez/ae3a17428bfec631d1b35dcdc6296a85#file-multi-label_classification_with_keras_imagedatagenerator-ipynbhttps://fairyonice.github.io/Part_5_Object_Detection_with_Yolo_using_VOC_2012_data_training.html
    - http://research.sualab.com/introduction/2017/11/29/image-recognition-overview-1.html